**Organic chemistry reaction prediction using neural machine translation (NMT).**

This seq2seq with attention uses Asynchronous Bidirectional Decoding with beam search for predicting reactions in SMILES format. 

In [ ]:
from keras.layers import Bidirectional, Concatenate, Dot, Input, Permute, Reshape
from keras.layers import RepeatVector, Dense, Activation, GRU, Lambda, Add
from keras import optimizers, regularizers, initializers
from keras.engine.topology import Layer
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
import random
from sklearn.model_selection import train_test_split

random.seed(40)

def preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty):
    
    X, Y = zip(*dataset)
    
    X = np.array([string_to_int(i, Tx, human_vocab) for i in X])
    Y = [string_to_int(t, Ty, machine_vocab) for t in Y]
    
    Xoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), X)))
    Yoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(machine_vocab)), Y)))

    return X, np.array(Y), Xoh, Yoh
        
        
def string_to_int(string, length, vocab):
    """
    Converts all strings in the vocabulary into a list of integers representing the positions of the
    input string's characters in the "vocab"
    
    Arguments:
    string -- input string
    length -- the number of time steps you'd like, determines if the output will be padded or cut
    vocab -- vocabulary, dictionary used to index every character of your "string"
    
    Returns:
    rep -- list of integers (or '<unk>') (size = length) representing the position of the string's character in the vocabulary
    """

    u = vocab["<unk>"]   
    if len(string) > length:
        string = string[:length]
        
    rep = list(map(lambda x: vocab.get(x, u), string))
    
    if len(string) < length:
        rep += [vocab['<pad>']] * (length - len(string))
    
    #print (rep)
    return rep


def int_to_string(ints, inv_vocab):
    """
    Output a machine readable list of characters based on a list of indexes in the machine's vocabulary
    
    Arguments:
    ints -- list of integers representing indexes in the machine's vocabulary
    inv_vocab -- dictionary mapping machine readable indexes to machine readable characters 
    
    Returns:
    l -- list of characters corresponding to the indexes of ints thanks to the inv_vocab mapping
    """
    
    l = [inv_vocab[i] for i in ints]
    return l


def softmax(x, axis=-1):
    """Softmax activation function.
    # Arguments
        x : Tensor.
        axis: Integer, axis along which the softmax normalization is applied.
    # Returns
        Tensor, output of softmax transformation.
    # Raises
        ValueError: In case `dim(x) == 1`.
    """
    ndim = K.ndim(x)
    if ndim == 2:
        return K.softmax(x)
    elif ndim > 2:
        e = K.exp(x - K.max(x, axis=axis, keepdims=True))
        s = K.sum(e, axis=axis, keepdims=True)
        return e / s
    else:
        raise ValueError('Cannot apply softmax to a tensor that is 1D')

Preprocessing the data:

In [ ]:
dataset = []
input_characters = set()
target_characters = set()

data_path = '../input/ocrtrain.csv'
lines = open(data_path).read().split('\n')

for line in lines[0: len(lines) - 1]:
    input_text, target_text = line.split(',')
    input_text = input_text[1:-2]
    input_text = input_text.split(' ')
    target_text = target_text.split(' ')

    if len(input_text)>=len(target_text) and len(input_text)<=15:
        ds = (input_text,target_text)
        dataset.append(ds)
        for char in input_text:
            if char not in input_characters:
                input_characters.add(char)
        for char in target_text:
            if char not in target_characters:
                target_characters.add(char)
                
z = np.array(dataset)
Tx = len(max(z[:,0], key=len))
Ty = len(max(z[:,1], key=len))
train, test = train_test_split(dataset, test_size=0.05, random_state = 43)
input_characters = sorted(list(input_characters)) + ['<unk>', '<pad>']
target_characters = sorted(list(target_characters)) + ['<unk>', '<pad>']
reactants_vocab = {v:k for k,v in enumerate(input_characters)}
products_vocab = {v:k for k,v in enumerate(target_characters)}
inv_products_vocab = {v:k for k,v in products_vocab.items()}

dataset = train
X, Y, Xoh, Yoh = preprocess_data(dataset, reactants_vocab, products_vocab, Tx, Ty)

print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("Xoh.shape:", Xoh.shape)
print("Yoh.shape:", Yoh.shape)

In [ ]:
index = 13
print("Reactants:", dataset[index][0])
print("Product:", dataset[index][1])
print()
print("Reactants after preprocessing (indices):", X[index])
print("Product after preprocessing (indices):", Y[index])
print()
print("Reactants after preprocessing (one-hot):", Xoh[index])
print("Product after preprocessing (one-hot):", Yoh[index])

Attention Layer for the forward Decoder:

In [ ]:
repeator = RepeatVector(1)
permutor = Permute((2,1))
dotor1 = Lambda(lambda x: K.batch_dot(x[0],x[1]))
activator = Activation('softmax')
dotor2 = Lambda(lambda x: K.batch_dot(x[0],x[1]))

def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-GRU.
    
    Arguments:
    a -- hidden state output of the Bi-GRU, numpy-array of shape (m, Tx, n_a)
    s_prev -- previous hidden state of the (post-attention) GRU, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attetion) GRU cell
    """
    
    s_prev = repeator(s_prev)
    a_trans = permutor(a)
    alphas = dotor1([s_prev,a_trans])
    alphas = activator(alphas)
    c = dotor2([alphas,a])
    
    return c

Attention Layer for the backward Decoder:

In [ ]:
repeator_rev = RepeatVector(1)
permutor_rev = Permute((2,1))
dotor1_rev = Lambda(lambda x: K.batch_dot(x[0],x[1]))
activator_rev = Activation('softmax')
dotor2_rev = Lambda(lambda x: K.batch_dot(x[0],x[1]))

def one_step_attention_rev(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-GRU.
    
    Arguments:
    a -- hidden state output of the Bi-GRU, numpy-array of shape (m, Tx, n_a)
    s_prev -- previous hidden state of the (post-attention) GRU, numpy-array of shape (m, n_a)
    
    Returns:
    context -- context vector, input of the next (post-attetion) GRU cell
    """
    
    s_prev = repeator_rev(s_prev)
    a_trans = permutor_rev(a)
    alphas = dotor1_rev([s_prev,a_trans])
    alphas = activator_rev(alphas)
    c = dotor2_rev([alphas,a])
    
    return c

Seq2Seq Model:
Slightly based on the model discussed in "Asynchronous Bidirectional Decoding for Neural Machine Translation" (https://arxiv.org/abs/1801.05122)


In [ ]:
n_a = 256
n_s = 256
batch_size = 128
post_activation_GRU_cell = GRU(n_s,dropout=0.1,recurrent_dropout=0.1)
post_activation_rev_GRU_cell = GRU(n_s,dropout=0.1,recurrent_dropout=0.1)
add_states1 = Add()
add_states2 = Add()
reshaper1 = Reshape((n_s,))
reshaper2 = Reshape((n_s,))
concatenator1 = Concatenate(axis=-1)
concatenator2 = Concatenate(axis=-1)
densor1 = Dense(n_s,activation='tanh')
densor2 = Dense(n_s,activation='tanh')
output_layer = Dense(len(products_vocab), activation='softmax',
                     bias_initializer=initializers.Constant(value=0.025),
                     activity_regularizer=regularizers.l2(0.05))


def model(Tx, Ty, n_a,n_s,reactants_vocab_size, products_vocab_size):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    human_vocab_size -- size of the python dictionary "human_vocab"
    machine_vocab_size -- size of the python dictionary "machine_vocab"
    Returns:
    model -- Keras model instance
    """
    
    X = Input(shape=(Tx, reactants_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    s = s0
    context_rev_seq = []
    av2_seq = []
    outputs = []
    a = Bidirectional(GRU(n_a,return_sequences=True,
                          recurrent_dropout=0.1),merge_mode='sum')(X)
    a_rev = Lambda(lambda x: K.reverse(x,axes=1),output_shape=(Tx,n_a,))(a)
    
    for t in range(Ty):
        context_rev = one_step_attention_rev(a_rev, s)
        s = post_activation_rev_GRU_cell(context_rev, initial_state = s)
        contextr = reshaper2(context_rev)
        c2 = concatenator2([contextr,s])
        av2 = densor2(c2)
        av2_seq.append(av2)
        context_rev_seq.append(context_rev)
    
    for t in range(Ty):
        context = one_step_attention(a, s)
        context_add = add_states1([context,context_rev_seq[Ty-1-t]])
        s = post_activation_GRU_cell(context_add, initial_state = s)
        context = reshaper1(context)
        c1 = concatenator1([context,s])
        av1 = densor1(c1)
        av = add_states2([av1,av2_seq[Ty-1-t]])
        out = output_layer(av)
        outputs.append(out)
        
    model = Model(inputs = [X,s0], outputs = outputs)
    return model


model = model(Tx, Ty, n_a,n_s, len(reactants_vocab), len(products_vocab))
model.summary()

In [ ]:
m = len(dataset)
s0 = np.zeros((m, n_s))
outputs = list(Yoh.swapaxes(0,1))

opt = optimizers.Adam(lr = 0.005, beta_1=0.9, beta_2=0.999, decay=0.0005)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
model.fit([Xoh,s0], outputs, batch_size = batch_size, epochs=130)
#model.save('model256.h5')
model.save_weights('weights256.h5')

In [ ]:
dataset = test
X, Y, Xoh, Yoh = preprocess_data(dataset, reactants_vocab, products_vocab, Tx, Ty)

In [ ]:
def beam_search_decoder(data, k=3):
    sequences = [[list(), 0.0]]
    # walk over each step in sequence
    for row in data:
        all_candidates = list()
        # expand each current candidate
        for i in range(len(sequences)):
            seq, score = sequences[i]
            for j in range(len(row)):
                candidate = [seq + [j], score-np.log(row[j])]
                all_candidates.append(candidate)
        # order all candidates by score
        ordered = sorted(all_candidates, key=lambda tup:tup[1])
        # select k best
        sequences = ordered[:k]
    return sequences

In [ ]:
output = []
pad = '<pad>'
m = len(dataset)
s0 = np.zeros((m, n_s))
bleu_score = np.zeros((m,1))
#prediction = model.predict([Xoh, s0, c0])
prediction = model.predict([Xoh, s0])
count = 0
bw = 5
for i in range(m):
    p0 = np.array(prediction)[:,i,:]
    p0 = beam_search_decoder(p0,bw)
    for j in reversed(range(bw)):
        p = p0[j][0]
        p = int_to_string(p,inv_products_vocab)
        o2 = []
        for x in p:
            if x != pad:
                o2.append(x)
        o1 = o2
        o2 = ''.join(o2)
        
        if o2 == ''.join(dataset[i][1]):
            count += 1
            bleu_score[i] = sentence_bleu([dataset[i][1]], o1)
            output.append(''.join(dataset[i][0])+','+''.join(dataset[i][1])+','+o2+','+str(o2 == ''.join(dataset[i][1])))
            break;
        elif j==0:
            bleu_score[i] = sentence_bleu([dataset[i][1]], o1)
            output.append(''.join(dataset[i][0])+','+''.join(dataset[i][1])+','+o2+','+str(o2 == ''.join(dataset[i][1])))

f = open('accuracy_bw.txt','w')
f.write(str(count/m))
f.close()

f = open('bleu_score_bw.txt','w')
f.write(str(sum(bleu_score)/m))
f.close()

print(count/m)
print(sum(bleu_score)/m)

with open('predicted_bw.csv','w') as file:
    for line in output:
        file.write(line)
        file.write('\n')

In [ ]:
output = []
pad = '<pad>'
m = len(dataset)
s0 = np.zeros((m, n_s))
bleu_score = np.zeros((m,1))
#prediction = model.predict([Xoh, s0, c0])
prediction = model.predict([Xoh, s0])
count = 0
for i in range(m):
    p = np.argmax(np.array(prediction)[:,i,:], axis = 1)
    p = int_to_string(p,inv_products_vocab)
    o2 = []
    for x in p:
        if x != pad:
            o2.append(x)
    bleu_score[i] = sentence_bleu([dataset[i][1]], o2)
    o2 = ''.join(o2)
    if o2 == ''.join(dataset[i][1]):
        count += 1
    output.append(''.join(dataset[i][0])+','+''.join(dataset[i][1])+','+o2+','+str(o2 == ''.join(dataset[i][1])))

print(count/m)
print(sum(bleu_score)/m)

f = open('accuracy.txt','w')
f.write(str(count/m))
f.close()

f = open('bleu_score.txt','w')
f.write(str(sum(bleu_score)/m))
f.close()

with open('predicted.csv','w') as file:
    for line in output:
        file.write(line)
        file.write('\n')